In [ ]:
# --- Imports (cell 1) ---
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Any, List, Tuple, Optional
from pathlib import Path
from datetime import datetime
import numpy as np
import pandas as pd
from flovopy.asl.wrappers import run_single_event, find_event_files

# Base project directories
PROJECTDIR = "/Users/GlennThompson/Dropbox/BRIEFCASE/SSADenver"
LOCALPROJECTDIR = "/Users/GlennThompson/work/PROJECTS/SSADenver_local"

# I/O
INPUT_DIR = f"{PROJECTDIR}/ASL_inputs/biggest_pdc_events"
OUTPUT_DIR = f"{LOCALPROJECTDIR}/asl_notebooks"
INVENTORY_XML = f"{PROJECTDIR}/MV.xml"
from obspy import read_inventory
inv = read_inventory(INVENTORY_XML)
print(f"[INV] Networks: {len(inv)}  Stations: {sum(len(n) for n in inv)}  Channels: {sum(len(sta) for net in inv for sta in net)}")

GLOBAL_CACHE: Optional[str] = f"{LOCALPROJECTDIR}/asl_global_cache"
GRIDFILE = f"{GLOBAL_CACHE}/Grid_530330bc.pkl"

from flovopy.asl.grid import Grid
gridobj = Grid.load(GRIDFILE)

'''
# Channel grid - avoid for now
from flovopy.asl.grid import nodegrid_from_channel_csvs, make_grid
GRID_KIND = "streams"   # Montserrat: stream-based NodeGrid
CHANNELS_DIR = f"{PROJECTDIR}/channel_finder/channels_csv"
CHANNELS_STEP_M = 100.0
CHANNELS_DEM_TIF = f"{PROJECTDIR}/channel_finder/02_dem_flipped_horizontal.tif"
REGULAR_GRID_DEM = None   # not used here
# --- Build grid object ---
if GRID_KIND.lower() == "streams":
    gridobj = nodegrid_from_channel_csvs(
        channels_dir=CHANNELS_DIR,
        step_m=CHANNELS_STEP_M,
        dem_tif=CHANNELS_DEM_TIF,
        approx_spacing_m=CHANNELS_STEP_M,
        max_points=None,
    )
    print(f"[GRID] NodeGrid: N={gridobj.node_lon.size}  approx_spacing_m={gridobj.approx_spacing_m}")
else:
    # Regular grid option (not used for this minimal cell, but left here for completeness)
    gridobj = make_grid(
        center_lat=16.72,  # Soufrière Hills dome approx (override if you prefer dome_locationfrom flovopy.core.mvo)
        center_lon=-62.18,
        node_spacing_m=NODE_SPACING_M,
        grid_size_lat_m=10_000,
        grid_size_lon_m=14_000,
        dem=None if REGULAR_GRID_DEM is None else (
            ("pygmt", {"resolution": REGULAR_GRID_DEM.split(":",1)[1], "cache_dir": f"{OUTPUT_DIR}/_dem_cache", "tag": REGULAR_GRID_DEM})
            if REGULAR_GRID_DEM.startswith("pygmt:")
            else ("geotiff", {"path": REGULAR_GRID_DEM.split(":",1)[1], "tag": Path(REGULAR_GRID_DEM.split(":",1)[1]).name})
        ),
    )
    print(f"[GRID] Regular Grid: shape={gridobj.nlat}x{gridobj.nlon}  spacing={gridobj.node_spacing_m} m")
DEM_TIF_FOR_BMAP = CHANNELS_DEM_TIF
'''

# ASL parameters
PEAKF = 8.0
METRIC = "VT"
WINDOW_SECONDS = 5
NODE_SPACING_M = 50
MIN_STATIONS = 5
REFINE_SECTOR = False   # enable triangular dome-to-sea refinement

# Physics / attenuation
WAVE_KIND = "surface"   # surface waves
DIST_MODE = "2d"        # include elevation
GRID_KIND = "regular"
SPEED = 1.5             # km/s
Q = 100
MISFIT_ENGINE = "l2"

# Basemap / region

DEM_TIF_FOR_BMAP = Path("/Users/glennthompson/Dropbox/PROFESSIONAL/DATA/wadgeDEMs/auto_crs_fit_v2/wgs84_s0.4_3_clean.tif")
REGION = (-62.255, -62.135, 16.66, 16.84)  # Montserrat default

# Limits
MAX_EVENTS: Optional[int] = 1  # cap number of events


In [ ]:
# --- Lightweight config object expected by run_single_event() (cell 2) ---
@dataclass
class LocalConfig:
    grid_kind: str = "regular"     # 'regular' or 'streams'
    wave_kind: str = "surface"     # 'surface' or 'body'
    dist_mode: str = "3d"          # '2d' or '3d'
    speed: float = 1.5             # km/s
    Q: int = 100
    misfit_engine: str = "l2"      # e.g. 'l2', 'huber'
    label: str = "nb_run"

    def tag(self) -> str:
        return f"{self.label}__G_{self.grid_kind}__W_{self.wave_kind}__D_{self.dist_mode}__V_{self.speed:g}__Q_{self.Q}__M_{self.misfit_engine}"

In [ ]:
# === Minimal grid → map → distances sanity check ===
from flovopy.asl.distances import compute_or_load_distances

# --- Output folder for this check ---
DBG_DIR = Path(OUTPUT_DIR) / f"test_asl_params"
DBG_DIR.mkdir(parents=True, exist_ok=True)

print("[SETUP] Writing artifacts to:", DBG_DIR)

# --- Distances (km) ---
use_3d = (DIST_MODE.lower() == "3d")
node_distances_km, station_coords, dist_meta = compute_or_load_distances(
    gridobj,
    inventory=inv,
    stream=None,
    cache_dir=str(DBG_DIR / "dist_cache"),
    force_recompute=False,           # force here, so this cell reflects current geometry
    use_elevation=use_3d,
)

print("[DIST] Meta:", dist_meta)

# Flatten all distances for global stats
all_d = np.concatenate([np.asarray(v, float).ravel() for v in node_distances_km.values()]) if node_distances_km else np.array([])
if all_d.size:
    finite = np.isfinite(all_d)
    d = all_d[finite]
    p = lambda q: float(np.nanpercentile(d, q))
    print(
        "[DIST] km  min={:.3f}  p50={:.3f}  p90={:.3f}  p95={:.3f}  max={:.3f}  N={}".format(
            float(np.nanmin(d)), float(np.nanmedian(d)), p(90), p(95), float(np.nanmax(d)), d.size
        )
    )

    # Per-station sanity: max distance per station id (helps catch unit mixups)
    per_sta_max = sorted(
        ((sid, float(np.nanmax(np.asarray(arr, float)))) for sid, arr in node_distances_km.items()),
        key=lambda x: x[1],
        reverse=True,
    )

    worst = per_sta_max[:5]
    print("[DIST] Worst per-station max distances (km):")
    for sid, mx in worst:
        print(f"  {sid:>16s}  max={mx:6.3f} km")

    # Simple sanity gate (Montserrat ~12 km island width; expect <= ~20 km)
    THRESH_KM = 20.0
    offenders = [(sid, mx) for sid, mx in per_sta_max if mx > THRESH_KM]
    if offenders:
        print(f"[WARN] {len(offenders)} station(s) exceed {THRESH_KM} km to some grid nodes (check units/CRS):")
        for sid, mx in offenders[:10]:
            sc = station_coords.get(sid, {})
            print(f"   {sid}  max={mx:.2f} km  @ ({sc.get('latitude'):.4f},{sc.get('longitude'):.4f}) elev={sc.get('elevation', np.nan)} m")
    else:
        print(f"[OK] All station→node distances ≤ {THRESH_KM} km (looks plausible for Montserrat).")

else:
    print("[DIST:ERR] No distances computed.")

In [ ]:
# --- Build the config object (cell 4) ---
cfg = LocalConfig(
    grid_kind=GRID_KIND,
    wave_kind=WAVE_KIND,
    dist_mode=DIST_MODE,
    speed=SPEED,
    Q=Q,
    misfit_engine=MISFIT_ENGINE,
    label="notebook_run",
)
cfg.tag()

In [ ]:
# --- Discover event files (cell 5) ---
input_dir = Path(INPUT_DIR)
#event_files = list(find_event_files(str(input_dir)))
event_files = list(find_event_files(input_dir))
print(f"Found {len(event_files)} event files.")
if MAX_EVENTS is not None:
    event_files = event_files[:MAX_EVENTS]
    print(f"Limiting to {len(event_files)} files.")
#event_files[:5]
for f in event_files:
    print(f)

In [ ]:
'''
# --- Run ASL per event (cell 6) ---
summaries: List[Dict[str, Any]] = []

for i, ev in enumerate(event_files, 1):
    print(f"[{i}/{len(event_files)}] {ev}")
    result = run_single_event(
        cfg,
        mseed_file=str(ev),
        inventory_xml=str(INVENTORY_XML),
        output_base=str(OUTPUT_DIR),
        node_spacing_m=NODE_SPACING_M,
        metric=METRIC,
        window_seconds=WINDOW_SECONDS,
        peakf=PEAKF,
        channels_dir=CHANNELS_DIR,
        channels_step_m=CHANNELS_STEP_M,
        channels_dem_tif=CHANNELS_DEM_TIF,
        regular_grid_dem=REGULAR_GRID_DEM,
        dem_tif_for_bmap=DEM_TIF_FOR_BMAP,
        simple_basemap=True,
        refine_sector=REFINE_SECTOR,
        region=REGION,
        MIN_STATIONS=MIN_STATIONS,
        GLOBAL_CACHE=GLOBAL_CACHE,
    )
    summaries.append(result)
'''

In [ ]:
# --- Run ASL per event (cell 6) ---
summaries: List[Dict[str, Any]] = []

for i, ev in enumerate(event_files, 1):
    print(f"[{i}/{len(event_files)}] {ev}")
    result = run_single_event(
        cfg,
        mseed_file=str(ev),
        inventory_xml=str(INVENTORY_XML),
        output_base=str(OUTPUT_DIR),
        node_spacing_m=NODE_SPACING_M,
        metric=METRIC,
        window_seconds=WINDOW_SECONDS,
        peakf=PEAKF,
        channels_dir=None,
        channels_step_m=None,
        channels_dem_tif=None,
        regular_grid_dem=str(DEM_TIF_FOR_BMAP),
        dem_tif_for_bmap=DEM_TIF_FOR_BMAP,
        simple_basemap=False,
        refine_sector=REFINE_SECTOR,
        region=REGION,
        MIN_STATIONS=MIN_STATIONS,
        GLOBAL_CACHE=GLOBAL_CACHE,
    )
    summaries.append(result)

In [ ]:
# --- Summarize results (cell 7) ---
df = pd.DataFrame(summaries)
display(df)

summary_csv = Path(OUTPUT_DIR) / f"{cfg.tag()}__summary.csv"
df.to_csv(summary_csv, index=False)
print(f"Summary saved to: {summary_csv}")

if not df.empty:
    n_ok = int((~df.get("error").notna()).sum()) if "error" in df.columns else len(df)
    print(f"Success: {n_ok}/{len(df)}")